In [6]:
import numpy as np 
import pandas as pd 
from prophet import Prophet
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
import datetime as dt
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
import statsmodels.api as sm
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima
std=StandardScaler()
import plotly.graph_objects as go


#pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [7]:
Dataset = pd.read_csv('number-of-new-hiv-infections-per-year.csv')
# Dataset.set_index('Year', inplace=True)
# Dataset['Year'] = pd.to_datetime(Dataset['Year'], format='%Y')
Dataset = Dataset[Dataset['Entity'] == 'World']

# Dataset
Dataset.drop(['Entity', 'Code'],axis=1,inplace=True)
Dataset.rename(columns = {'Year':'ds', 'Adults (ages 15+) and children (ages 0-14) newly infected with HIV':'y'}, inplace = True)
Dataset['y'] = Dataset['y'].cumsum()
Dataset.dtypes
Dataset.shape


(31, 2)

In [8]:
fig = px.line(Dataset, x="ds", y="y")
fig.show()

In [9]:
# Dataset=Dataset.query("ds< 1979")

# Dataset

In [10]:
model_compare=Dataset.iloc[50:]
error_compare = pd.DataFrame({'Model': pd.Series(dtype='str'),
                   'RMSE': pd.Series(dtype='float64'),'MSE': pd.Series(dtype='float64'),'MAPE': pd.Series(dtype='float64'),'R2 Score': pd.Series(dtype='float64')})
error_compare

,Model,RMSE,MSE,MAPE,R2 Score


# Linear Regression

In [11]:
train_ml=Dataset.iloc[:50]
valid_ml=Dataset.iloc[50:]
model_scores=[]

In [12]:
lin_reg=LinearRegression(normalize=True)

In [13]:
lin_reg.fit(np.array(train_ml["ds"]).reshape(-1,1),np.array(train_ml["y"]).reshape(-1,1))

C:\Users\anure\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)





LinearRegression(normalize=True)

In [14]:
prediction_valid_linreg=lin_reg.predict(np.array(valid_ml["ds"]).reshape(-1,1))

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by LinearRegression.

In [ ]:
model_scores.append(np.sqrt(mean_squared_error(valid_ml["y"],prediction_valid_linreg)))
print("Root Mean Square Error for Linear Regression: ",np.sqrt(mean_squared_error(valid_ml["y"],prediction_valid_linreg)))

In [15]:
model_compare["Linear Regression"]= prediction_valid_linreg
model_compare
# prediction_valid_linreg

NameError: name 'prediction_valid_linreg' is not defined

In [22]:
new_row = {'Model':'Linear Regression', 'RMSE': np.sqrt(mean_squared_error(valid_ml["y"],prediction_valid_linreg))}
#append row to the dataframe
error_compare = error_compare.append(new_row, ignore_index=True)
error_compare



C:\Users\anure\AppData\Local\Temp\ipykernel_20496\3324062175.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Model,RMSE
0,Linear Regression,6.230489e+06


In [23]:

plt.figure(figsize=(11,6))
prediction_linreg=lin_reg.predict(np.array(Dataset["ds"]).reshape(-1,1))
linreg_output=[]
for i in range(prediction_linreg.shape[0]):
    linreg_output.append(prediction_linreg[i][0])

fig=go.Figure()
fig.add_trace(go.Scatter(x=Dataset["ds"], y=Dataset["y"],
                    mode='lines+markers',name="Train Data for Anomaly"))
fig.add_trace(go.Scatter(x=Dataset["ds"], y=linreg_output,
                    mode='lines',name="Linear Regression Best Fit Line",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Linear Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

<Figure size 792x432 with 0 Axes>

# Polynomial Regression

In [24]:
train_ml=Dataset.iloc[:50]
valid_ml=Dataset.iloc[50:]

In [25]:
poly = PolynomialFeatures(degree = 4) 

In [26]:
train_poly=poly.fit_transform(np.array(train_ml["ds"]).reshape(-1,1))
valid_poly=poly.fit_transform(np.array(valid_ml["ds"]).reshape(-1,1))
y=train_ml["y"]

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by PolynomialFeatures.

In [27]:
linreg=LinearRegression(normalize=True)
linreg.fit(train_poly,y)

NameError: name 'y' is not defined

In [28]:
prediction_poly=linreg.predict(valid_poly)
rmse_poly=np.sqrt(mean_squared_error(valid_ml["y"],prediction_poly))
model_scores.append(rmse_poly)
print("Root Mean Squared Error for Polynomial Regression: ",rmse_poly)

NameError: name 'valid_poly' is not defined

In [29]:
new_row = {'Model':'Polynomial Regression', 'RMSE': model_scores[1]}
error_compare = error_compare.append(new_row, ignore_index=True)
error_compare


IndexError: list index out of range

In [30]:
comp_data=poly.fit_transform(np.array(Dataset["ds"]).reshape(-1,1))
plt.figure(figsize=(11,6))
predictions_poly=linreg.predict(comp_data)

fig=go.Figure()
fig.add_trace(go.Scatter(x=Dataset["ds"], y=Dataset["y"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=Dataset["ds"], y=predictions_poly,
                    mode='lines',name="Polynomial Regression Best Fit",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Polynomial Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",
                 legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

<Figure size 792x432 with 0 Axes>

In [31]:
Dataset.dtypes

ds    int64
y     int64
dtype: object

In [32]:
new_prediction_poly=[]
for i in range(1,18):
    new_date_poly=poly.fit_transform(np.array(Dataset["ds"].max()+i).reshape(-1,1))
    new_prediction_poly.append(linreg.predict(new_date_poly)[0])

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# SVM

In [33]:
# svm=SVR(C=1,degree=6,kernel='poly',epsilon=0.01)

In [34]:
#Fitting model on the training data
# svm.fit(np.array(train_ml["Year"]).reshape(-1,1),np.array(train_ml["Anomaly"]).reshape(-1,1))

In [35]:

# prediction_valid_svm=svm.predict(np.array(valid_ml["Year"]).reshape(-1,1))

In [36]:
Dataset.dtypes

Dataset.dtypes




# new_date=[]
# new_prediction_lr=[]
# new_prediction_svm=[]
# for i in range(1,18):
#     new_date.append(Dataset.index[-1]+timedelta(year=i))
#     new_prediction_lr.append(lin_reg.predict(np.array(Dataset.index.max()+i).reshape(-1,1))[0][0])
# #     new_prediction_svm.append(svm.predict(np.array(datewise["Days Since"].max()+i).reshape(-1,1))[0])



# pd.set_option('display.float_format', lambda x: '%.6f' % x)
# model_predictions=pd.DataFrame(zip(new_date,new_prediction_lr,new_prediction_poly),
#                                columns=["Dates","Linear Regression Prediction","Polynonmial Regression Prediction"])
# model_predictions.head()

ds    int64
y     int64
dtype: object

# Time Series Forecasting 

## Holt's Linear Model

In [35]:
model_train=Dataset.iloc[:25]
valid=Dataset.iloc[25:]
y_pred=valid.copy()
valid

,ds,y
3970,2015,64100000
3971,2016,65800000
3972,2017,67500000
3973,2018,69100000
3974,2019,70600000
3975,2020,72100000


In [36]:
holt=Holt(np.asarray(model_train["y"])).fit(optimized=True)   

C:\Users\anure\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



In [37]:
y_pred["Holt"]=holt.forecast(len(valid))
model_scores.append(np.sqrt(mean_squared_error(y_pred["y"],y_pred["Holt"])))
print("Root Mean Square Error Holt's Linear Model: ",np.sqrt(mean_squared_error(y_pred["y"],y_pred["Holt"])))

Root Mean Square Error Holt's Linear Model:  987794.9184008773


In [38]:
model_compare["Holt's Linear Model"]= y_pred["Holt"]
model_compare


C:\Users\anure\AppData\Local\Temp\ipykernel_7620\1648763570.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,y,Holt's Linear Model
3970,NaN,NaN,6.422056e+07
3971,NaN,NaN,6.614007e+07
3972,NaN,NaN,6.805958e+07
3973,NaN,NaN,6.997909e+07
3974,NaN,NaN,7.189861e+07
3975,NaN,NaN,7.381812e+07


In [39]:
new_row = {'Model':'Holt\'s Linear Model', 'RMSE': model_scores[2]}
error_compare = error_compare.append(new_row, ignore_index=True)
error_compare

IndexError: list index out of range

In [40]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train["ds"], y=model_train["y"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid["ds"], y=valid["y"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid["ds"], y=y_pred["Holt"],
                    mode='lines+markers',name="Prediction of Confirmed Cases",))
fig.update_layout(title="Confirmed Cases Holt's Linear Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

# AR Model (using AUTO ARIMA)

In [41]:
model_train=Dataset.iloc[:25]
valid=Dataset.iloc[25:]
y_pred=valid.copy()
valid

,ds,y
3970,2015,64100000
3971,2016,65800000
3972,2017,67500000
3973,2018,69100000
3974,2019,70600000
3975,2020,72100000


In [42]:
model_ar= auto_arima(model_train["y"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=4,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar.fit(model_train["y"])

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=599.761, Time=0.03 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=572.693, Time=0.02 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=575.297, Time=0.06 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=575.265, Time=0.10 sec
 ARIMA(4,2,0)(0,0,0)[0] intercept   : AIC=565.804, Time=0.17 sec

Best model:  ARIMA(4,2,0)(0,0,0)[0] intercept
Total fit time: 0.412 seconds


ARIMA(order=(4, 2, 0), scoring_args={}, suppress_warnings=True)

In [43]:
def ad_test(dataset):
    dftest=adfuller(dataset,autolag='AIC')
    print("1. ADF: ",dftest[0])
    print("2. P-Value: ",dftest[1])
    print("3. No. of Lags: ",dftest[2])
    print("4. Num of Observations Used for ADF Regression and Critical Value Calculation: ",dftest[3])
    print("5. Critical Values: ")
    for key, val in dftest[4].items():
        print("\t", key, ": ",val)

In [44]:
ad_test(model_train['y'])

1. ADF:  -5.378862051594775
2. P-Value:  3.7512353839156806e-06
3. No. of Lags:  0
4. Num of Observations Used for ADF Regression and Critical Value Calculation:  24
5. Critical Values: 
	 1% :  -3.7377092158564813
	 5% :  -2.9922162731481485
	 10% :  -2.635746736111111


In [45]:
prediction_ar=model_ar.predict(len(valid))
y_pred["AR Model Prediction"]=prediction_ar

In [46]:
model_scores.append(np.sqrt(mean_squared_error(y_pred["y"],y_pred["AR Model Prediction"])))
print("Root Mean Square Error for AR Model: ",np.sqrt(mean_squared_error(y_pred["y"],y_pred["AR Model Prediction"])))

Root Mean Square Error for AR Model:  513907.2133816084


In [47]:
model_compare["AR Model Prediction"]= y_pred["AR Model Prediction"]
model_compare



C:\Users\anure\AppData\Local\Temp\ipykernel_7620\2341629467.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,y,Holt's Linear Model,AR Model Prediction
3970,NaN,NaN,6.422056e+07,6.401620e+07
3971,NaN,NaN,6.614007e+07,6.563872e+07
3972,NaN,NaN,6.805958e+07,6.716913e+07
3973,NaN,NaN,6.997909e+07,6.860812e+07
3974,NaN,NaN,7.189861e+07,6.995646e+07
3975,NaN,NaN,7.381812e+07,7.121343e+07


In [48]:
new_row = {'Model': 'AR Model Prediction', 'RMSE': model_scores[3]}
error_compare = error_compare.append(new_row, ignore_index=True)
error_compare

IndexError: list index out of range

In [49]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train["ds"], y=model_train["y"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid["ds"], y=valid["y"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid["ds"], y=y_pred["AR Model Prediction"],
                    mode='lines+markers',name="Prediction of Confirmed Cases",))
fig.update_layout(title="Confirmed Cases AR Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [50]:
model_predictions = pd.DataFrame()
AR_model_new_prediction=[]
for i in range(1,18):
    AR_model_new_prediction.append(model_ar.predict(len(valid)+i)[-1])
model_predictions["AR Model Prediction"]=AR_model_new_prediction
model_predictions

,AR Model Prediction
0,7.237931e+07
1,7.345420e+07
2,7.443812e+07
3,7.533105e+07
4,7.613302e+07
5,7.684405e+07
6,7.746414e+07
7,7.799330e+07
8,7.843152e+07
9,7.877882e+07


# Fb-Prophet

In [51]:
model_train=Dataset.iloc[:45]
valid=Dataset.iloc[45:]
model_train.rename(columns = {'Year':'ds', 'Anomaly':'y'}, inplace = True)
valid.rename(columns = {'Year':'ds', 'Anomaly':'y'}, inplace = True)
# model_train

fb_prophet = Prophet(interval_width = 0.95, yearly_seasonality = True)
fb_prophet.fit(model_train)

C:\Users\anure\AppData\Local\Temp\ipykernel_7620\1594898381.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

11:41:29 - cmdstanpy - INFO - Chain [1] start processing
11:41:30 - cmdstanpy - INFO - Chain [1] done processing


In [52]:
in_forecast = fb_prophet.predict(valid)
in_forecast['ds'] = in_forecast['ds'].apply(lambda x: x.year)
in_forecast

ValueError: Dataframe has no rows.

In [ ]:
Dataset_prophet = Dataset.copy()

Dataset_prophet.rename(columns = {'Year':'ds', 'Anomaly':'y'}, inplace = True)



in_result_df = Dataset_prophet.merge(in_forecast, on='ds', how='left')

fig = px.line(
    in_result_df,
    x='ds',
    y=['y', 'yhat'],
    color_discrete_sequence=['black', 'red'],
    labels={'ds': "Year"},
    title="Average Global Temperature Change Over Time"
)

fig.show()

In [88]:
future_df = pd.DataFrame(np.arange(2022, 2031), columns=['ds'])
future_df

,ds
0,2022
1,2023
2,2024
3,2025
4,2026
5,2027
6,2028
7,2029
8,2030


In [89]:
out_model = Prophet(interval_width = 0.95, yearly_seasonality = True)
out_model.fit(Dataset_prophet)

13:04:44 - cmdstanpy - INFO - Chain [1] start processing
13:04:44 - cmdstanpy - INFO - Chain [1] done processing


In [90]:
out_forecast = out_model.predict(future_df)
out_forecast['ds'] = out_forecast['ds'].apply(lambda x: x.year)
out_forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1970,-266639.317175,-92602.173257,330719.210904,-266639.317175,-266639.317175,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,0.0,0.0,0.0,110463.67786
1,1970,-266639.317175,-78341.949939,325825.169852,-266639.317175,-266639.317175,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,0.0,0.0,0.0,110463.67786
2,1970,-266639.317175,-91714.668919,318717.374528,-266639.317175,-266639.317175,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,0.0,0.0,0.0,110463.67786
3,1970,-266639.317175,-87338.455226,325637.827535,-266639.317175,-266639.317175,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,0.0,0.0,0.0,110463.67786
4,1970,-266639.317175,-90820.611506,312772.796419,-266639.317175,-266639.317175,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,0.0,0.0,0.0,110463.67786
5,1970,-266639.317175,-106802.338054,317969.072556,-266639.317175,-266639.317175,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,0.0,0.0,0.0,110463.67786
6,1970,-266639.317175,-91824.445634,309270.021992,-266639.317175,-266639.317175,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,0.0,0.0,0.0,110463.67786
7,1970,-266639.317175,-91208.430015,330287.924666,-266639.317175,-266639.317175,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,0.0,0.0,0.0,110463.67786
8,1970,-266639.317175,-85943.134838,324484.499481,-266639.317175,-266639.317175,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,377102.995035,0.0,0.0,0.0,110463.67786


In [53]:
import plotly.express as px
out_result_df = pd.concat([Dataset_prophet, out_forecast], axis=0)
fig = px.line(
    out_result_df,
    x='ds',
    y=['y', 'yhat'],
    color_discrete_sequence=['black', 'red'],
    labels={'ds': "Year"},
    title="Average Global Temperature Change Over Time"
)

fig.show()

NameError: name 'Dataset_prophet' is not defined

In [54]:
tempdf=in_forecast[['ds','yhat']]

tempdf.rename(columns = {'ds':'ds', 'yhat':'FB-Prophet Prediction'}, inplace = True)



model_compare=model_compare.merge(tempdf)

# model_compare.rename(columns = {'yhat':'FB-Prophet Prediction'}, inplace = True)

# model_compare.head()

model_compare

# model_compare["AR Model Prediction"]= y_pred["AR Model Prediction"]
# model_compare

NameError: name 'in_forecast' is not defined

In [55]:
new_row2 = {'Model':'FB-Prophet', 'RMSE': np.sqrt(mean_squared_error(model_compare["y"],model_compare["FB-Prophet Prediction"]))}
#append row to the dataframe
error_compare = error_compare.append(new_row2, ignore_index=True)
error_compare

KeyError: 'FB-Prophet Prediction'

# SARIMA Model (using AUTO ARIMA)

In [56]:
Train = Dataset.iloc[:45]
Valid = Dataset.iloc[45:]
Y_pred_sarima = valid.copy()

In [57]:
model_sarima= auto_arima(Train["y"],trace=True, error_action='ignore', 
                         start_p=0,start_q=0,max_p=2,max_q=2,m=7,
                   suppress_warnings=True,stepwise=True,seasonal=True)
model_sarima.fit(Train["y"])

Performing stepwise search to minimize aic
 ARIMA(0,2,0)(1,0,1)[7]             : AIC=inf, Time=0.36 sec
 ARIMA(0,2,0)(0,0,0)[7]             : AIC=751.096, Time=0.02 sec
 ARIMA(1,2,0)(1,0,0)[7]             : AIC=728.088, Time=0.05 sec
 ARIMA(0,2,1)(0,0,1)[7]             : AIC=751.053, Time=0.07 sec
 ARIMA(1,2,0)(0,0,0)[7]             : AIC=726.237, Time=0.02 sec
 ARIMA(1,2,0)(0,0,1)[7]             : AIC=728.081, Time=0.06 sec
 ARIMA(1,2,0)(1,0,1)[7]             : AIC=730.009, Time=0.24 sec
 ARIMA(2,2,0)(0,0,0)[7]             : AIC=728.178, Time=0.05 sec
 ARIMA(1,2,1)(0,0,0)[7]             : AIC=729.798, Time=0.06 sec
 ARIMA(0,2,1)(0,0,0)[7]             : AIC=749.054, Time=0.02 sec
 ARIMA(2,2,1)(0,0,0)[7]             : AIC=728.768, Time=0.10 sec
 ARIMA(1,2,0)(0,0,0)[7] intercept   : AIC=726.203, Time=0.02 sec
 ARIMA(1,2,0)(1,0,0)[7] intercept   : AIC=728.193, Time=0.08 sec
 ARIMA(1,2,0)(0,0,1)[7] intercept   : AIC=728.194, Time=0.09 sec
 ARIMA(1,2,0)(1,0,1)[7] intercept   : AIC=730.180, 

ARIMA(order=(1, 2, 0), scoring_args={}, seasonal_order=(0, 0, 0, 7),
      suppress_warnings=True)

In [58]:
prediction_sarima=model_sarima.predict(len(Valid))
Y_pred_sarima["SARIMA Model Prediction"]=prediction_sarima

ValueError: Prediction must have `end` after `start`.

In [ ]:
# model_scores.append(np.sqrt(mean_squared_error(Y_pred_sarima["Anomaly"],Y_pred_sarima["SARIMA Model Prediction"])))
# print("Root Mean Square Error for SARIMA Model: ",np.sqrt(mean_squared_error(Y_pred_sarima["Anomaly"],Y_pred_sarima["SARIMA Model Prediction"])))





In [108]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train["ds"], y=model_train["y"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid["ds"], y=valid["y"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid["ds"], y=Y_pred_sarima["SARIMA Model Prediction"],
                    mode='lines+markers',name="Prediction for Confirmed Cases",))
fig.update_layout(title="Confirmed Cases SARIMA Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [109]:
model_compare["SARIMA"] = Y_pred_sarima["SARIMA Model Prediction"]
model_compare


,ds,y,Linear Regression,Holt's Linear Model,AR Model Prediction,FB-Prophet Prediction,SARIMA
0,1965,112280,156404.131313,73062.575909,74885.571296,182841.091554,NaN
1,1966,92804,152975.704172,62759.638326,65738.376713,163127.958467,NaN
2,1967,131697,149547.277031,52456.700742,84728.642306,143406.693770,NaN
3,1968,80024,146118.849890,42153.763159,77721.478806,123697.203987,NaN
4,1969,54202,142690.422749,31850.825575,75159.082795,169363.071610,NaN
5,1970,33706,139261.995608,21547.887992,65137.290214,149649.938523,NaN
6,1971,52806,135833.568467,11244.950409,66070.596689,129928.673826,NaN
7,1972,65153,132405.141326,942.012825,64232.378458,110219.184043,NaN
8,1973,135859,128976.714185,-9360.924758,64273.962670,155885.051666,NaN
9,1974,218367,125548.287044,-19663.862341,60315.890892,136171.918579,NaN


In [ ]:
new_row = {'Model': 'SARIMA Model Prediction', 'RMSE': np.sqrt(mean_squared_error(y_pred["Anomaly"],y_pred["SARIMA Model Prediction"]))}
error_compare = error_compare.append(new_row, ignore_index=True)
error_compare
# 